# 🌐 AgentCore Browser with Web Auth Signing 

## Overview

In this tutorial we will learn how to enable Web Bot Auth signing with Amazon Bedrock AgentCore Browser tool. 

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands                                                                          |
| LLM model           | Claude 4.5 Haiku                                                                 |
| Tutorial components | Browser automation, Web Bot Auth signing of browser Requests                     |
| Tutorial vertical   |                                                                                  |
| Example complexity  | Intermediate                                                                     |
| SDK used            | Amazon BedrockAgentCore Python SDK, Strands Agents, Strands Agent Tools          |
 
### Tutorial Architecture


### Tutorial Key Features

* Using browser tool in a headless way with Strands agents
* AgentCoreBrowser integration using strands_tools.browser
* Claude 4.5 Haiku model for fast and efficient analysis

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials configured
* Amazon Bedrock AgentCore SDK
* Strands agents and strands-agents-tools packages
* Access to Claude 4.5 Haiku model in Amazon Bedrock

**Implementation**: Using the official `strands_tools.browser.AgentCoreBrowser` for seamless browser automation with intelligent financial analysis.

## 🎯 What This Does
- **AgentCoreBrowser Integration** using `strands_tools.browser`
- **Structured Invoke Pattern** for implementations
- **Claude 4.5 Haiku Model** for fast and efficient analysis

## 🔐 Setup: Create Execution Role

Before creating the browser configuration, we need to create an IAM execution role that grants the AgentCore Browser the necessary permissions to operate. This role will allow the browser to:

- Make signed requests to AWS services
- Access network resources
- Write logs and metrics
- Generate authentication tokens

We'll use the `create_agentcore_role` utility function from the tutorials folder to create this role with all required permissions.

In [ ]:
# Import required libraries
import boto3
import json
import time
import os
# Get account information
accountId = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

print(f"Account ID: {accountId}")
print(f"Region: {region}")

# Create IAM client
iam_client = boto3.client("iam")

# Define role name for browser signing
browser_role_name = "agentcore-browser-signing-role"

# Define the trust policy for browser signing
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": accountId
                },
                "ArnLike": {
                    "aws:SourceArn": [
                        f"arn:aws:bedrock-agentcore:{region}:{accountId}:*"
                        
                    ]
                }
            }
        }
    ]
}

# Define minimal permissions policy
permissions_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "MinimalBrowserPermissions",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": f"arn:aws:logs:{region}:{accountId}:log-group:/aws/bedrock-agentcore/browser/*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:ListMultipartUploadParts",
                "s3:AbortMultipartUpload"
            ],
            "Resource": "arn:aws:s3:::YOUR_S3_FOLDER/replay-data/*",
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": account_id
                }
            }
        }
    ]
}

print(f"\n🔐 Creating minimal execution role for AgentCore Browser with signing support...")

# Create or update the IAM role
try:
    # Try to create the role
    role_response = iam_client.create_role(
        RoleName=browser_role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="Minimal role for AgentCore Browser with signing capabilities"
    )
    print(f"✅ Role created: {browser_role_name}")
    
    # Wait for role to be available
    time.sleep(10)
    
except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"⚠️  Role {browser_role_name} already exists - updating trust policy...")
    
    # Update the trust policy
    iam_client.update_assume_role_policy(
        RoleName=browser_role_name,
        PolicyDocument=json.dumps(trust_policy)
    )
    
    # Delete existing inline policies
    try:
        policies = iam_client.list_role_policies(RoleName=browser_role_name)
        for policy_name in policies.get('PolicyNames', []):
            iam_client.delete_role_policy(RoleName=browser_role_name, PolicyName=policy_name)
    except Exception as e:
        print(f"   Warning: Could not clean up old policies: {e}")
    
    print(f"✅ Role updated: {browser_role_name}")
    time.sleep(5)

# Attach the minimal permissions policy
try:
    iam_client.put_role_policy(
        RoleName=browser_role_name,
        PolicyName="MinimalBrowserPermissions",
        PolicyDocument=json.dumps(permissions_policy)
    )
    print("✅ Minimal permissions policy attached")
except Exception as e:
    print(f"⚠️  Warning: Could not attach permissions policy: {e}")

# Get the role ARN
execution_role_arn = f"arn:aws:iam::{accountId}:role/{browser_role_name}"

print(f"\n✅ Role Configuration Complete:")
print(f"   Role Name: {browser_role_name}")
print(f"   Role ARN: {execution_role_arn}")
print(f"\n🔑 Trust Policy includes:")
print(f"   • bedrock-agentcore.amazonaws.com (standard)")
print(f"   • preprod.genesis-service.aws.internal (for browser signing)")
print(f"\n⏳ Waiting for IAM permissions to propagate...")
time.sleep(10)
print("✅ Ready to create browser with signing enabled")

## 🔧 AgentCore Browser Configuration

Before using the AgentCore Browser tool, you can optionally create a custom browser configuration with specific settings like recording capabilities, network configuration, and execution roles. This section shows how to create a custom browser configuration using the AWS SDK.

### Create Custom Browser Configuration

The following code demonstrates how to create a custom AgentCore Browser with recording enabled and a specific execution role:

#### 📚 Create the Browser Configuration with Web Bot Auth Signing

**What is Browser Signing?**

Browser signing (`browserSigning.enabled = True`) configures the AgentCore Browser to automatically sign all outgoing HTTP requests. This is essential for:

- **Authenticated API access**: Sign requests to protected APIs
- **AWS service integration**: Automatically sign requests with AWS credentials
- **Security compliance**: Ensure all browser requests are authenticated
- **Request integrity**: Cryptographically sign request headers

When enabled, the browser will:
1. Intercept all HTTP/HTTPS requests
2. Add cryptographic signatures to request headers
3. Include authentication tokens automatically
4. Maintain session integrity across requests

**Note**: This uses internal AWS endpoints (gamma) for development/testing. In production, use standard endpoints.

In [ ]:
import boto3
import uuid

cp_client = boto3.client('bedrock-agentcore-control', 
                         region_name='us-west-2')

dp_client = boto3.client('bedrock-agentcore',
                         region_name='us-west-2')

print(f"Account ID: {accountId}")
print(f"Region: {region}")
print(f"Using Execution Role: {execution_role_arn}")

response = cp_client.create_browser(
    name="web_bot_auth_browser_" + str(uuid.uuid4())[:6],
    description="Browser configured to sign web bot auth",
    networkConfiguration={
        "networkMode": "PUBLIC"
    },
    executionRoleArn=execution_role_arn,
    browserSigning={
        "enabled": True
    }
)

browserId = response['browserId']
browserArn = response['browserArn']
print(f"\n✅ Browser Created Successfully!")
print(f"   Browser ID: {browserId}")
print(f"   Browser ARN: {browserArn}")
print(f"\n🔐 Browser signing is ENABLED - all requests will be automatically signed")

In [ ]:
from strands import Agent
from strands_tools.browser import AgentCoreBrowser
import asyncio
import time

print("🚀 Using native async/await - no nest_asyncio needed!")

# Create and configure the Strands agent with AgentCoreBrowser
# Initialize the official AgentCoreBrowser tool with custom browser ID
agent_core_browser = AgentCoreBrowser(identifier=browserId, region="us-west-2")
    
# Create agent with Claude 4.5 Haiku model and optimized settings
strands_agent = Agent(
    tools=[agent_core_browser.browser],
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    system_prompt="""You are a website analyst with browser signing capabilities.
1. Use the browser tool to visit and interact with the website EFFICIENTLY
2. Focus on extracting key information QUICKLY and within 2-3 browser interactions.
3. All requests will be signed automatically via browser signing configuration."""
)

print("\n🚀 Starting analysis with browser signing enabled...")

# Define async function
async def analyze_website(agent, prompt):
    """Async wrapper for agent invocation"""
    try:
        # Use the async method directly
        result = await agent.invoke_async(prompt)
        return result
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return None
    
# Use top-level await (works natively in Jupyter/IPython 7.0+)
result = await analyze_website(
    strands_agent,
    "Please analyze the website at https://www.amazon.com/ and provide key insights"
)

if result:
    print("\n✅ Analysis completed!")
    print(result)

## 🎭 What Happened Behind the Scene

When you execute this notebook with **browser signing enabled**, the following process occurs:

### 1. **Browser Configuration Creation**
```python
browserSigning={
    "enabled": True
}
```
This tells the AgentCore Browser service to automatically sign all HTTP requests made by the browser.

### 2. **Agent Initialization**
The Strands agent is initialized with:
- Custom browser ID (with signing enabled)
- Claude 4.5 Haiku model
- Browser tool integration

### 3. **Request Signing Flow**
When the agent navigates to websites:

```
Agent Request → AgentCore Browser → Sign Request Headers → Target Website
                    ↓
            Add Authentication Headers:
            - Authorization signatures
            - Timestamp headers
            - Security tokens
            - Request checksums
```

## 🔒 Security Benefits

With browser signing enabled:

✅ **Automatic Authentication** - No manual header management  
✅ **Request Integrity** - Cryptographic signatures prevent tampering  
✅ **AWS Integration** - Seamless IAM credential integration  
✅ **Session Management** - Secure session token handling  
✅ **Audit Trail** - All signed requests are logged  

## 🛠️ Troubleshooting

### Error: "Browser session not found"
**Solution**: The browser ID might have expired. Re-run the browser creation cell (cell-3)
